<a href="https://colab.research.google.com/github/LeonardoVieiraGuimaraes/MiniCursoPalestra/blob/main/redeNeuralArtificial/ObjetoQueda/QuedaLivre_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Queda Livre — demonstração e rede neural (Colab)

Este notebook demonstra a fórmula de queda livre e treina uma pequena rede neural para aprender a relação entre tempo (t) e distância (s). Foi preparado para rodar no Google Colab ou em Jupyter local.

## Fórmula física

Para queda livre sem resistência do ar e com aceleração da gravidade `g`, a posição `s(t)` é:

s(t) = s0 + v0 * t + 0.5 * g * t^2

Neste notebook usamos `s0 = 0` e `v0 = 0`, portanto `s(t) = 0.5 * g * t^2` com `g = 9.81 m/s^2`.

In [ ]:
# Célula 1 — preparar ambiente no Colab de forma segura
# Evitamos instalar TensorFlow/numpy para não sobrescrever versões pré-instaladas no Colab
IN_COLAB = False
try:
    import google.colab
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    print('Executando no Colab. Usaremos as bibliotecas pré-instaladas para evitar conflitos.')
    try:
        import tensorflow as tf
        print('TensorFlow disponível — versão:', tf.__version__)
    except Exception:
        print('TensorFlow não encontrado no runtime. Se necessário, instale uma versão compatível manualmente.')
    # atualizar apenas pacotes não-críticos (matplotlib)
    try:
        get_ipython().system('pip -q install -U matplotlib')
    except Exception:
        pass
else:
    print('Não detectado Colab — para execução local certifique-se de ter um ambiente virtual e instale dependências:')
    print('  python -m venv .venv')
    print(r'  .venv\Scripts\Activate.ps1  (PowerShell)')
    print('  pip install numpy matplotlib tensorflow')

### 1) Preparação do ambiente

Propósito: detectar se estamos executando no Google Colab e preparar o ambiente de forma segura, sem reinstalar pacotes críticos (TensorFlow/numpy) que podem causar conflitos no runtime.

Entradas: nenhuma.

Saídas: variáveis no namespace (`IN_COLAB`) e mensagens informativas sobre o ambiente; atualização de pacotes não-críticos (opcional).

Notas de debug:
- Se o Colab não tiver TensorFlow, a célula informa como instalar manualmente.
- Em execução local, instrui sobre criação de virtualenv e instalação de dependências.


In [ ]:
# Célula 2 — gerar dataset (t -> s)
import numpy as np
def free_fall_distance(t, g=9.81, v0=0.0, s0=0.0):
    return s0 + v0 * t + 0.5 * g * (t ** 2)

def generate_dataset(n_samples=5000, t_min=0.0, t_max=10.0, noise_std=0.0, seed=42):
    rng = np.random.default_rng(seed)
    t = rng.uniform(t_min, t_max, size=(n_samples, 1)).astype(np.float32)
    s = free_fall_distance(t).astype(np.float32)
    if noise_std > 0:
        s += rng.normal(0, noise_std, size=s.shape).astype(np.float32)
    return t, s

t_train, s_train = generate_dataset(5000, 0.0, 10.0, noise_std=0.0, seed=1)
t_test, s_test = generate_dataset(1000, 0.0, 10.0, noise_std=0.0, seed=2)

import os
os.makedirs('data', exist_ok=True)
np.savez_compressed('data/dataset.npz', t_train=t_train, s_train=s_train, t_test=t_test, s_test=s_test)
print('Dataset salvo em data/dataset.npz — exemplos:')
print('t[0]=', t_train[0,0], 's[0]=', s_train[0,0])

Gerar arquivo de inferência (CSV/NPZ):
Esta célula cria um arquivo `data/times_for_infer.csv` contendo uma coluna `t` com 100 valores uniformes entre 0 e 10s. Também salva `data/dataset_infer.npz` com `t` e `s_true` para referência/validação.

In [ ]:
# Célula adicional — cria arquivos para inferência em lote
import numpy as np
import os
os.makedirs('data', exist_ok=True)
# 100 tempos uniformes entre 0 e 10s
t_infer = np.linspace(0.0, 10.0, num=100, dtype=np.float32).reshape(-1,1)
# calcula a posição verdadeira sem ruído para referência
def free_fall_distance(t, g=9.81, v0=0.0, s0=0.0):
    return s0 + v0 * t + 0.5 * g * (t ** 2)
s_true = free_fall_distance(t_infer)
# salvar CSV com coluna 't' (compatível com QuedaLivre_infer_colab upload CSV)
import pandas as pd
df = pd.DataFrame({'t': t_infer.flatten()})
csv_path = 'data/times_for_infer.csv'
df.to_csv(csv_path, index=False)
# salvar npz com t e s_true
np.savez_compressed('data/dataset_infer.npz', t=t_infer, s_true=s_true)
print('Arquivos gerados:')
print(' -', csv_path)
print(
)

### 2) Geração do dataset

Propósito: gerar um dataset sintético mapeando tempo `t` para posição `s(t)` usando a fórmula da queda livre. Suporta adicionar ruído nas medições para simular imperfeições.

Entradas: parâmetros de geração — número de amostras, intervalo de tempo (t_min, t_max), desvio padrão do ruído, semente RNG.

Saídas: arrays `t_train`, `s_train`, `t_test`, `s_test`, e arquivo salvo `data/dataset.npz`.

Dicas:
- Use `seed` para reprodutibilidade.
- Se quiser incluir arrasto, substitua a função analítica por um integrador numérico (ex.: RK4) e re-gere o dataset.


In [ ]:
# Célula 3 — treinar um modelo Keras para mapear t -> s
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

data = np.load('data/dataset.npz')
t_train = data['t_train']
s_train = data['s_train']
t_test = data['t_test']
s_test = data['s_test']

# normalização simples
t_mean, t_std = t_train.mean(), t_train.std()
s_mean, s_std = s_train.mean(), s_train.std()
t_train_n = (t_train - t_mean) / t_std
t_test_n = (t_test - t_mean) / t_std
s_train_n = (s_train - s_mean) / s_std
s_test_n = (s_test - s_mean) / s_std

def build_model():
    model = models.Sequential([
        layers.Input(shape=(1,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='linear')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

model = build_model()
model.summary()
history = model.fit(t_train_n, s_train_n, epochs=60, batch_size=64, validation_split=0.1, verbose=2)

# avaliar
test_loss, test_mae = model.evaluate(t_test_n, s_test_n, verbose=0)
print(f'Teste - MSE: {test_loss:.6f}, MAE: {test_mae:.6f}')

os.makedirs('model', exist_ok=True)
# Salvar usando a extensão recomendada .keras para compatibilidade com Keras
model.save('model/fall_model.keras')
np.savez('model/scaler.npz', t_mean=t_mean, t_std=t_std, s_mean=s_mean, s_std=s_std)
print('Modelo e scalers salvos em model/')

# plot do histórico
plt.figure(figsize=(6,4))
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.grid(True)
plt.show()

### 3) Treinamento do modelo

Propósito: carregar o dataset salvo, normalizar as variáveis, definir e treinar um modelo Keras (MLP) que mapeia `t -> s`.

Entradas: arquivo `data/dataset.npz` gerado anteriormente.

Saídas: modelo treinado salvo em `model/fall_model`, scalers salvos em `model/scaler.npz`, e gráficos do histórico de treino.

Dicas:
- Normalize `t` e `s` para melhorar convergência do treinamento.
- Ajuste `epochs`, `batch_size` e arquitetura para balancear erro e tempo de execução.


In [ ]:
# Célula 4 — uso do modelo treinado: prever s a partir de t
from tensorflow.keras.models import load_model
import numpy as np
from google.colab import files

mdl = load_model('model/fall_model')
sc = np.load('model/scaler.npz')
t_mean, t_std = sc['t_mean'], sc['t_std']
s_mean, s_std = sc['s_mean'], sc['s_std']

def predict_from_time(t_seconds):
    t_arr = np.array([[float(t_seconds)]], dtype=np.float32)
    t_n = (t_arr - t_mean) / t_std
    s_n = mdl.predict(t_n)
    s = s_n * s_std + s_mean
    return float(s[0,0])

# exemplo interativo
for t in [0.5, 1.0, 2.0, 3.0]:
    print(t, 's -> s=', predict_from_time(t))

### 4) Inferência e uso do modelo

Propósito: demonstrar como carregar o modelo e scalers salvos e usar o modelo para prever `s` a partir de `t` (predição pontual e em lote).

Entradas: `model/fall_model` e `model/scaler.npz`.

Saídas: funções utilitárias (`predict_from_time`) e exemplos de uso imprimindo previsões.

Dicas:
- Se for usar no Colab e quiser persistir o modelo, monte o Google Drive e copie `model/` para `drive/MyDrive/...`.
- Para inverter (estimativa de `t` a partir de `s`), gere um dataset com `s` como entrada e treine um novo modelo.


---
### Notas finais
- Para salvar o modelo no seu Google Drive monte o Drive e copie a pasta `model/` para `/content/drive/MyDrive/...`
- Se quiser estimar t a partir de s, gere dataset invertido (s como entrada) e treine similarmente.